## Sequence number rewriting

In [ ]:
import sys
sys.path.insert(0, "../ext")
import plot_utils as pu
import random
from sys import exit
from itertools import batched
from collections import Counter

### Generate packets for a flow with the semantics of frames and video quality layers

In [ ]:
PACKETS_COUNT = 10000
MIN_PACKETS_PER_FRAME = 3
MAX_PACKETS_PER_FRAME = 7
NUM_LAYERS = 2

In [ ]:
random.seed(42)
START_ORIG_SEQNUM = random.randint(1, 1000)
frames_at_source = []
current_packet_count = 0
group_count = 1
frame_count = 1

# Packet format: [frame_group_number, frame_number, layer_number, sequence_number, packet_start_interm_end_mark, if lost then False else True]

while True:
    for layer in range(1, NUM_LAYERS+1):
        frame = []
        packet_count_in_frame = random.randint(MIN_PACKETS_PER_FRAME, MAX_PACKETS_PER_FRAME)
        for seqnum in range(current_packet_count, current_packet_count+packet_count_in_frame):
            frame.append([group_count, frame_count, layer, START_ORIG_SEQNUM+seqnum, 2, True])
        frame[0][-2] = 1
        frame[-1][-2] = 3
        current_packet_count += packet_count_in_frame
        frames_at_source.append(frame)
        frame_count += 1
    group_count += 1
    if current_packet_count > PACKETS_COUNT:
        break

In [ ]:
print("Groups:")
for gr in batched(frames_at_source, 2):
    print(gr)

In [ ]:
# Convert frames to packets
packets_at_source = []
for frame in frames_at_source:
    packets_at_source += frame

In [ ]:
print("Packets at source:\n")
for p in packets_at_source:
    print(p)

### Drop quality layers and rewrite sequence numbers to mask it

In [ ]:
DROP_LAYERS = [2]

In [ ]:
packets_at_source_baseline = []
drop_count_at_source_baseline = 0

for gn, fn, ln, sn, mk, pr in packets_at_source:
    if ln in DROP_LAYERS:
        drop_count_at_source_baseline += 1
        packets_at_source_baseline.append([gn, fn, ln, sn, -1, mk, pr])
    else:
        packets_at_source_baseline.append([gn, fn, ln, sn, sn-drop_count_at_source_baseline, mk, pr])

In [ ]:
print("Packets at source baseline:\n")
for p in packets_at_source_baseline:
    print(p)

### Simulate loss and reordering of packets

In [ ]:
def simulate_loss_and_reordering(param_loss_prob, param_reorder_prob, param_reorder_max_skip):

    if param_loss_prob + param_reorder_prob > 1.0:
        print("Error: Loss + reordering prob exceeds 1.0")
        exit(0)

    random.seed(42)
    packets_in = []
    packets_lost = []
    packets_reordered = []
    
    reorder_skip_set = False
    reorder_skip_count = 0
    
    for gn, fn, ln, sn, rn, mk, pr in packets_at_source_baseline:
    
        # Handle reordering
        if reorder_skip_set:
            packets_reordered.append([])
            reorder_skip_set = False
        if reorder_skip_count > 0:
            packets_reordered[-1].append([gn, fn, ln, sn, rn, mk, pr])
            packets_in.insert(len(packets_in)-1, [gn, fn, ln, sn, rn, mk, pr])
            reorder_skip_count -= 1
        else:
            dice_roll = random.randint(1, 100)/100
            if dice_roll <= param_loss_prob:
                packets_lost.append([gn, fn, ln, sn, rn, mk, pr])
                packets_in.append([gn, fn, ln, sn, rn, mk, False])
            elif dice_roll > param_loss_prob and dice_roll <= (param_loss_prob + param_reorder_prob):
                reorder_skip_set = True
                reorder_skip_count = random.randint(1, param_reorder_max_skip)
                packets_in.append([gn, fn, ln, sn, rn, mk, pr])
            else:
                packets_in.append([gn, fn, ln, sn, rn, mk, pr])

    print(f"Desired loss rate: {param_loss_prob}, Actual loss rate: {round(len(packets_lost)/PACKETS_COUNT, 2)}")
    print(f"Desired reordering rate: {param_reorder_prob}, Actual reordering rate: {round(len(packets_reordered)/PACKETS_COUNT, 2)}")

    return packets_in, packets_lost, packets_reordered

### Simulate sequence number rewriting solution

In [ ]:
# Solution types:
# 0 = Only maintain cumulative intentional drop count (D)
# 1 = Perfect solution, involves buffering packets when unsure
# 2 = Maintain invariants
# 3 = Frame-aware, data plane-aware
# 4 = Exactly what the Tofino sees (sees no suppressed packet)
# 5 = S-LR (same as 4)
# 6 = S-LM

In [ ]:
def solution_type0(packets_in):
    
    packets_out_ = []
    drop_count = 0
    
    for gn, fn, ln, sn, rn, mk, pr in packets_in:
        if not pr:
            continue
        if ln in DROP_LAYERS:
            drop_count += 1
            packets_out_.append([gn, fn, ln, sn, rn, -1, mk])
        else:
            packets_out_.append([gn, fn, ln, sn, rn, sn-drop_count, mk])

    return packets_out

In [ ]:
def solution_type1(packets_in):
    
    packets_out_ = []
    drop_counts = []
    seq_space = []
    frame_types = []
    i = 0

    for gn, fn, ln, sn, rn, mk, pr in packets_in:

        if not pr:
            continue

        found = False
        where = -1 # 0 = left, 1 = in, 2 = right
        continuous_left = False
        continuous_right = False
        for i, seq_range in enumerate(seq_space):
            if sn < seq_range[0]:
                found = True
                where = 0
                if sn+1 == seq_range[0]:
                    continuous_left = True
                break
            elif sn >= seq_range[0] and sn <= seq_range[1]:
                found = True
                where = 1
                break
            elif sn > seq_range[1]:
                found = True
                where = 2
                if seq_range[1]+1 == sn:
                    continuous_right = True

        cumulative_drop_count = 0
        stop_index = i
        if where == 2:
            stop_index += 1

        # Accumulate known drop counts
        for j in range(stop_index):
            cumulative_drop_count += drop_counts[j]

        # Figure out drop counts for missing packets
        last_frame_type = None
        for j in range(stop_index):
            next_frame_type = frame_types[j][0]
            # Has frame changed
            if last_frame_type is None or last_frame_type[0] != next_frame_type[0]:
                pass
            else:
                pass
            last_frame_type = next_frame_type
        
        if found is False:
            seq_space.append([sn, sn])
            drop_counts.append(0)
            frame_types.append([(fn, ln, mk), (fn, ln, mk)])
        else:
            if where == 0 and continuous_left and continuous_right:
                seq_space = seq_space[:i-1] + [[seq_space[i-1][0], seq_space[i][1]]] + seq_space[i+1:]
                drop_counts = drop_counts[:i-1] + [drop_counts[i-1] + drop_counts[i]] + drop_counts[i+1:]
                frame_types = frame_types[:i-1] + [[frame_types[i-1][0], frame_types[i][1]]] + frame_types[i+1:]
            elif where == 0 and continuous_left:
                seq_space[i][0] = sn
                frame_types[i][0] = (fn, ln, mk)
            elif where == 0 and continuous_right:
                seq_space[i-1][1] = sn
                frame_types[i-1][1] = (fn, ln, mk)
            elif where == 0:
                seq_space = seq_space[:i] + [[sn, sn]] + seq_space[i:]
                drop_counts = drop_counts[:i] + [0] + drop_counts[i:]
                frame_types = frame_types[:i] + [[(fn, ln, mk), (fn, ln, mk)]] + frame_types[i:]
            elif where == 1:
                print(f"Packet: {[gn, fn, ln, sn, rn, mk]}, Retransmission")
            elif where == 2 and continuous_right:
                seq_space[-1][1] = sn
                frame_types[-1][1] = (fn, ln, mk)
            elif where == 2:
                seq_space.append([sn ,sn])
                drop_counts.append(0)
                frame_types.append([(fn, ln, mk), (fn, ln, mk)])

        # Check if we know the layer of the hole
        if where == 0 and not continuous_left:
            pass
        elif where == 2 and not continuous_right:
            pass
        
        if ln in DROP_LAYERS:
            for i in range(len(seq_space)):
                if sn >= seq_space[i][0] and sn <= seq_space[i][1]:
                    drop_counts[i] += 1
                    break
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk]
        else:
            packet_out_ = [gn, fn, ln, sn, rn, sn-cumulative_drop_count, mk]
        packets_out_.append(packet_out_)

        print(f"Packet: {packet_out_}, sequence space: {seq_space}, drop counts: {drop_counts}, frame types: {frame_types}")

    return packets_out_

In [ ]:
def solution_type2(packets_in):
    
    packets_out_ = []
    max_sn = 0
    drop_count = 0
    
    for gn, fn, ln, sn, rn, mk, pr in packets_in:

        if not pr:
            continue
        
        if max_sn == 0 and sn > 0:
            max_sn = sn - 1

        if sn == max_sn + 1:
            # Regular case: Continuous sequence number
            if ln in DROP_LAYERS:
                packet_out_ = [gn, fn, ln, sn, rn, -1, mk]
                drop_count += 1
            else:
                srn = sn - drop_count
                packet_out_ = [gn, fn, ln, sn, rn, srn, mk]
            max_sn = sn
            
        elif sn > max_sn + 1:
            # Skips sequence numbers
            if ln in DROP_LAYERS:
                packet_out_ = [gn, fn, ln, sn, rn, -1, mk]
            else:
                srn = sn - drop_count
                packet_out_ = [gn, fn, ln, sn, rn, srn, mk]
            max_sn = sn
            
        elif sn <= max_sn:
            # Filling a hole
            if ln in DROP_LAYERS:
                packet_out_ = [gn, fn, ln, sn, rn, -1, mk]
            else:
                srn = max_sn - drop_count - 1
                packet_out_ = [gn, fn, ln, sn, rn, srn, mk]
        
        packets_out_.append(packet_out_)
        
        print(f"Packet out: {packet_out_}, drop count: {drop_count}, max sequence number: {max_sn}")
                
    return packets_out_

In [ ]:
def solution_type3(packets_in, drop_when_unsure=False):
    
    packets_out_ = []
    drop_count = 0
    max_frame_number = 0
    max_frame_layer = 0
    max_frame_start_sn = 0
    max_frame_max_sn = 0
    max_frame_end_seen = False
    max_dropped_frame = 0
    SKIP_CADENCE = 2
    
    for gn, fn, ln, sn, rn, mk, pr in packets_in:

        if not pr:
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk, False]
            packets_out_.append(packet_out_)
            continue

        # First packet
        if fn > 1 and max_frame_number == 0:
            max_frame_number = fn
            max_frame_layer = ln
            if mk == 1:
                max_frame_start_sn = sn
            else:
                # If the first packet is not the starting packet of a frame, assume the previous packet was
                max_frame_start_sn = sn - 1
            if mk == 3:
                max_frame_max_sn = sn
            else:
                max_frame_max_sn = sn + 1
            
            if ln not in DROP_LAYERS:
                srn = sn
            else:
                srn = -1
                if mk == 3:
                    drop_count = 2
            if mk == 3:
                max_frame_end_seen = True

        # In the same frame
        elif fn == max_frame_number:
            if mk == 1:
                max_frame_start_sn = sn
            elif sn == max_frame_start_sn:
                # This packet is not the starting packet but is set equal to max starting sequence number
                max_frame_start_sn = sn - 1
            if mk == 3:
                max_frame_max_sn = sn
            elif not max_frame_end_seen:
                max_frame_max_sn = sn + 1
            # print(f"max_frame_number: {max_frame}, max_frame_start_sn: {max_frame_start_sn}, max_frame_max_sn: {max_frame_max_sn}")
                
            if ln not in DROP_LAYERS:
                srn = sn - drop_count
            else:
                srn = -1
                if mk == 3:
                    drop_count += sn - max_frame_start_sn + 1
                    # print(f"Here:: sn - max_frame_start_sn + 1 = {sn - max_frame_start_sn + 1}")
            if mk == 3:
                max_frame_end_seen = True

        # Moved to the next consecutive frame
        elif fn == max_frame_number + 1:
            max_frame_number = fn
            if max_frame_layer in DROP_LAYERS and not max_frame_end_seen:
                # Last frame was supposed to be dropped but the last packet wasn't seen and so drop count wasn't adjusted
                if mk == 1:
                    drop_count += sn - max_frame_start_sn
                else:
                    # print(f"max_frame_max_sn: {max_frame_max_sn}, sn-2: {sn-2}, max_frame_start_sn: {max_frame_start_sn}")
                    # drop_count += max(max_frame_max_sn, sn-2) - max_frame_start_sn + 1
                    drop_count += 3
            elif ln in DROP_LAYERS and mk == 3:
                if max_frame_end_seen:
                    drop_count += sn - max_frame_max_sn
                else:
                    drop_count += 2
                
            max_frame_layer = ln
                
            if mk == 1:
                max_frame_start_sn = sn
            else:
                max_frame_start_sn = sn - 1
            if mk == 3:
                max_frame_max_sn = sn
            else:
                max_frame_max_sn = sn + 1

            if ln not in DROP_LAYERS:
                srn = sn - drop_count
            else:
                srn = -1
            if mk == 3:
                max_frame_end_seen = True
            else:
                max_frame_end_seen = False

        # Skipped to a future frame
        elif fn > max_frame_number + 1:

            ## OPTIMIZATION: If a skipped frame was supposed to be dropped and we don't have any other information, increment drop count by 2 packets
            if max_frame_layer not in DROP_LAYERS and ln not in DROP_LAYERS and fn - max_frame_number + 1 >= SKIP_CADENCE:
                drop_count += 2

            max_frame_number = fn

            ## OPTIMIZATION: Last frame was supposed to be dropped but the last packet wasn't seen and so drop count wasn't adjusted
            if max_frame_layer in DROP_LAYERS and not max_frame_end_seen:
                drop_count += max_frame_max_sn - max_frame_start_sn

            max_frame_layer = ln

            if mk == 1:
                max_frame_start_sn = sn
            elif sn == max_frame_start_sn:
                max_frame_start_sn = sn - 1
            elif mk == 2:
                max_frame_start_sn = sn - 1
            elif mk == 3:
                max_frame_start_sn = sn - 2
                
            if mk == 3:
                max_frame_max_sn = sn
            else:
                max_frame_max_sn = sn + 1

            if ln not in DROP_LAYERS:
                srn = sn - drop_count
            else:
                srn = -1
                if mk == 3:
                    drop_count += 2
            if mk == 3:
                max_frame_end_seen = True
            else:
                max_frame_end_seen = False

        # Skipped to a past frame
        elif fn < max_frame_number:
            if ln in DROP_LAYERS:
                srn = -1
            elif fn + 1 == max_frame_number:
                # Immediate next frame is the max seen
                if max_frame_layer not in DROP_LAYERS:
                    # No adjustments have been made to drop count since next frame is not a dropped layer 
                    srn = sn - drop_count
                elif not max_frame_end_seen:
                    # No adjustments have been made to drop count since next frame isn't over yet
                    srn = sn - drop_count
                else:
                    # Drop count needs to be adjusted before application
                    srn = sn - (drop_count - (max_frame_max_sn - max_frame_start_sn + 1))
                    # print(f"Here2: drop_count - (max_frame_max_sn - max_frame_start_sn + 1) = {drop_count - (max_frame_max_sn - max_frame_start_sn + 1)}")
            elif fn > max_dropped_frame:
                # Can trust drop count because it hasn't been adjusted after this frame
                srn = sn - drop_count
            else:
                # We don't know anymore
                if drop_when_unsure:
                    srn = -1
                else:
                    srn = sn - drop_count

        # Adjust max dropped frame number
        if ln in DROP_LAYERS:
            max_dropped_frame = max(fn, max_dropped_frame)
        
        packet_out_ = [gn, fn, ln, sn, rn, srn, mk, True]
        packets_out_.append(packet_out_)
        
        # print(f"Packet out: {packet_out_}, drop count: {drop_count}, max frame: {max_frame}, max frame layer: {max_frame_layer},"
        #       + f" max frame start: {max_frame_start_sn}, max frame max: {max_frame_max_sn}, max frame end seen: {max_frame_end_seen}")
                
    return packets_out_

## Hardware-amenable algorithms

### Stateless

In [ ]:
# Packet format: [frame_group_number, frame_number, layer_number, sequence_number, packet_start_interm_end_mark, if lost then False else True]
# Constants: [skip_cadence, skip_set_frames_count]
# Metadata: [frame_number, sequence_number, rewritten_sequence_number,]
# Obtained directly from packet headers: [frame_number, layer_number, sequence_number, packet_start_interm_end_mark]
# Obtained indirectly from packet headers: [max_dropped_frame]
# State per receive stream in SR-LR: [max_frame_number, max_frame_start_sequence_number, max_frame_max_sequence_number,
#                            max_frame_end_seen, drop_count]
# State per receive stream in SR-LM: [max_frame_number, max_frame_max_sequence_number, drop_count]

def solution_type4(packets_in):
    
    packets_out_ = []
    drop_count = 0
    max_frame_number = 0
    max_dropped_frame = 0
    SKIP_CADENCE = 2
    SKIP_SET_FRAMES_COUNT = 1
    
    for gn, fn, ln, sn, rn, mk, pr in packets_in:

        # If packet is lost, it's not seen by the algorithm
        if not pr:
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk, False]
            packets_out_.append(packet_out_)
            continue

        # If packet is suppressed during replication, it's not seen by the algorithm
        if fn % SKIP_CADENCE == 0:
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk, True]
            packets_out_.append(packet_out_)
            continue

        # Obtained from a match-action table in the control plane
        max_dropped_frame = (fn // SKIP_CADENCE) * SKIP_CADENCE

        # First packet
        if fn > 1 and max_frame_number == 0:
            max_frame_number = fn
            srn = sn

        # In the same frame
        elif fn == max_frame_number:
            srn = sn - drop_count

        # Moved to the next consecutive frame
        elif fn == max_frame_number + 1:
            max_frame_number = fn
            srn = sn - drop_count

        # Skipped to a future frame
        elif fn > max_frame_number + 1:
            # Only one set of dropped frames have been encountered
            if fn - max_frame_number == SKIP_CADENCE:
                # Most conservative estimate of drop_count
                drop_count += 3 * SKIP_SET_FRAMES_COUNT
            # Multiple sets of dropped frames have been encountered
            else:
                num_dropped_sets = (fn - max_frame_number)//SKIP_CADENCE
                # Most conservative estimate of drop_count
                drop_count += 3 * num_dropped_sets * SKIP_SET_FRAMES_COUNT
            max_frame_number = fn
            srn = sn - drop_count

        # Skipped to a past frame
        elif fn < max_frame_number:
            # No updates to state
            srn = sn - drop_count
        
        packet_out_ = [gn, fn, ln, sn, rn, srn, mk, True]
        packets_out_.append(packet_out_)
        
        # print(f"Packet out: {packet_out_}, drop count: {drop_count}, max frame: {max_frame}, max frame layer: {max_frame_layer},"
        #       + f" max frame start: {max_frame_start_sn}, max frame max: {max_frame_max_sn}, max frame end seen: {max_frame_end_seen}")
                
    return packets_out_

### Lower memory, higher retx. penalty implementation

In [ ]:
# Packet format: [frame_group_number, frame_number, layer_number, sequence_number, packet_start_interm_end_mark, if lost then False else True]
# Constants: [skip_cadence, skip_set_frames_count]
# Metadata: [frame_number, sequence_number, rewritten_sequence_number,]
# Obtained directly from packet headers: [frame_number, layer_number, sequence_number, packet_start_interm_end_mark]
# Obtained indirectly from packet headers: [max_dropped_frame]
# State per receive stream in SR-LR: [max_frame_number, max_frame_start_sequence_number, max_frame_max_sequence_number,
#                            max_frame_end_seen, drop_count]
# State per receive stream in SR-LM: [max_frame_number, max_frame_max_sequence_number, drop_count]

def solution_type5(packets_in):
    
    packets_out_ = []
    drop_count = 0
    max_frame_number = 0
    SKIP_CADENCE = 2
    SKIP_SET_FRAMES_COUNT = 1
    
    for gn, fn, ln, sn, rn, mk, pr in packets_in:

        # If packet is lost, it's not seen by the algorithm
        if not pr:
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk, False]
            packets_out_.append(packet_out_)
            continue

        # If packet is suppressed during replication, it's not seen by the algorithm
        if fn % SKIP_CADENCE == 0:
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk, True]
            packets_out_.append(packet_out_)
            continue

        # First packet
        if fn > 1 and max_frame_number == 0:
            max_frame_number = fn
            srn = sn

        # In the same frame
        elif fn == max_frame_number:
            srn = sn - drop_count

        # Moved to the next consecutive frame
        elif fn == max_frame_number + 1:
            max_frame_number = fn
            srn = sn - drop_count

        # Skipped to a future frame
        elif fn > max_frame_number + 1:
            # Only one set of dropped frames have been encountered
            if fn - max_frame_number == SKIP_CADENCE:
                # Most conservative estimate of drop_count
                drop_count += 3 * SKIP_SET_FRAMES_COUNT
            # Multiple sets of dropped frames have been encountered
            else:
                num_dropped_sets = (fn - max_frame_number)//SKIP_CADENCE
                # Most conservative estimate of drop_count
                drop_count += 3 * num_dropped_sets * SKIP_SET_FRAMES_COUNT
            max_frame_number = fn
            srn = sn - drop_count

        # Skipped to a past frame
        elif fn < max_frame_number:
            # No updates to state
            srn = sn - drop_count
        
        packet_out_ = [gn, fn, ln, sn, rn, srn, mk, True]
        packets_out_.append(packet_out_)
        
        # print(f"Packet out: {packet_out_}, drop count: {drop_count}, max frame: {max_frame}, max frame layer: {max_frame_layer},"
        #       + f" max frame start: {max_frame_start_sn}, max frame max: {max_frame_max_sn}, max frame end seen: {max_frame_end_seen}")
                
    return packets_out_

In [ ]:
# Packet format: [frame_group_number, frame_number, layer_number, sequence_number, packet_start_interm_end_mark, if lost then False else True]
# Constants: [skip_cadence, skip_set_frames_count]
# Metadata: [frame_number, sequence_number, rewritten_sequence_number,]
# State per receive stream: [max_frame_number, max_frame_start_sequence_number, max_frame_max_sequence_number,
#                            max_frame_end_seen, drop_count, max_dropped_frame]

def solution_type6(packets_in, drop_when_unsure=False):
    
    packets_out_ = []
    drop_count = 0
    max_frame_number = 0
    # max_frame_layer = 0
    max_frame_start_sn = 0
    max_frame_max_sn = 0
    max_frame_end_seen = False
    max_dropped_frame = 0
    SKIP_CADENCE = 2
    SKIP_SET_FRAMES_COUNT = 1
    
    for gn, fn, ln, sn, rn, mk, pr in packets_in:

        # If packet is lost, it's not seen by the algorithm
        if not pr:
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk, False]
            packets_out_.append(packet_out_)
            continue

        # If packet is suppressed during replication, it's not seen by the algorithm
        if fn % SKIP_CADENCE == 0:
            packet_out_ = [gn, fn, ln, sn, rn, -1, mk, True]
            packets_out_.append(packet_out_)
            continue

        max_dropped_frame = (fn // SKIP_CADENCE) * SKIP_CADENCE

        # First packet
        if fn > 1 and max_frame_number == 0:
            max_frame_number = fn
            
            if mk == 1:
                max_frame_start_sn = sn
            else:
                # If the first packet is not the starting packet of a frame, assume the previous packet was
                max_frame_start_sn = sn - 1
            
            if mk == 3:
                max_frame_max_sn = sn
            else:
                max_frame_max_sn = sn + 1
            
            srn = sn
            
            if mk == 3:
                max_frame_end_seen = True

        # In the same frame
        elif fn == max_frame_number:
            
            if mk == 1:
                max_frame_start_sn = sn
            elif sn == max_frame_start_sn:
                # This packet is not the starting packet but is set equal to max starting sequence number
                max_frame_start_sn = sn - 1
            
            if mk == 3:
                max_frame_max_sn = sn
            elif not max_frame_end_seen:
                max_frame_max_sn = sn + 1
            # print(f"max_frame_number: {max_frame}, max_frame_start_sn: {max_frame_start_sn}, max_frame_max_sn: {max_frame_max_sn}")
            
            srn = sn - drop_count
            
            if mk == 3:
                max_frame_end_seen = True

        # Moved to the next consecutive frame
        elif fn == max_frame_number + 1:
            
            max_frame_number = fn
                
            if mk == 1:
                max_frame_start_sn = sn
            else:
                max_frame_start_sn = sn - 1
            if mk == 3:
                max_frame_max_sn = sn
            else:
                max_frame_max_sn = sn + 1

            srn = sn - drop_count
            
            if mk == 3:
                max_frame_end_seen = True
            else:
                max_frame_end_seen = False

        # Skipped to a future frame
        elif fn > max_frame_number + 1:

            # Only one set of dropped frames have been encountered
            if fn - max_frame_number == SKIP_CADENCE:
                
                # We saw the last packet of the previous retained frame and the first packet of the next retained frame
                if max_frame_end_seen and mk == 1:
                    drop_count += sn - max_frame_max_sn - 1
                
                # We either didn't see the last packet of the previous retained frame or didn't see the first packet of the next retained frame
                # Unsure, so guessing the most conservative drop_count
                else:
                    drop_count += 3 * SKIP_SET_FRAMES_COUNT

            # Multiple sets of dropped frames have been encountered
            else:

                num_dropped_sets = (fn - max_frame_number)//SKIP_CADENCE

                # Most conservative estimate of drop_count
                drop_count += 3 * num_dropped_sets * SKIP_SET_FRAMES_COUNT

            max_frame_number = fn

            if mk == 1:
                max_frame_start_sn = sn
            elif sn == max_frame_start_sn:
                max_frame_start_sn = sn - 1
            elif mk == 2:
                max_frame_start_sn = sn - 1
            elif mk == 3:
                max_frame_start_sn = sn - 2
                
            if mk == 3:
                max_frame_max_sn = sn
            else:
                max_frame_max_sn = sn + 1

            srn = sn - drop_count

            if mk == 3:
                max_frame_end_seen = True
            else:
                max_frame_end_seen = False

        # Skipped to a past frame
        elif fn < max_frame_number:
            
            if fn + 1 == max_frame_number:
                # No adjustments have been made to drop count
                srn = sn - drop_count
            
            elif fn > max_dropped_frame:
                # Can trust drop count because it hasn't been adjusted after this frame
                srn = sn - drop_count
                
            else:
                # We don't know anymore
                if drop_when_unsure:
                    srn = -1
                else:
                    srn = sn - drop_count
        
        packet_out_ = [gn, fn, ln, sn, rn, srn, mk, True]
        packets_out_.append(packet_out_)
        
        # print(f"Packet out: {packet_out_}, drop count: {drop_count}, max frame: {max_frame}, max frame layer: {max_frame_layer},"
        #       + f" max frame start: {max_frame_start_sn}, max frame max: {max_frame_max_sn}, max frame end seen: {max_frame_end_seen}")
                
    return packets_out_

## Hardware-amenable sequence number-rewriting heuristics used in Scallop

### Sequence Rewriting - Low Memory (SR-LM)

In [ ]:
def scallop_sr_lm(packets_in):
    return solution_type5(packets_in)

### Sequence Rewriting - Low Retransmission (SR-LR)

In [ ]:
def scallop_sr_lr(packets_in):
    return solution_type6(packets_in, drop_when_unsure=False)

## Validation

In [ ]:
def report_sanity_check_results(packets_in, packets_out):
    if len([p for p in packets_in if p[6]]) != len([p for p in packets_out if p[7]]):
        print("ERROR: All packets that went in didn't come out!")

In [ ]:
def compute_extra_drops(packets_out):
    extra_drops = 0
    for gn, fn, ln, sn, rn, srn, mk, pr in packets_out:
        if pr and rn > -1 and srn == -1:
            extra_drops += 1
    print(f"Extra drops: {extra_drops}")
    return extra_drops*100.0/PACKETS_COUNT

In [ ]:
def compute_extra_retransmissions(packets_out, exit_on_error=True):

    def count_skips(seqnums):
        skips = 0
        last_nonzero_seqnum = -1
        for seqnum in seqnums:
            if last_nonzero_seqnum > -1:
                skips += seqnum - last_nonzero_seqnum - 1
            last_nonzero_seqnum = seqnum
        return skips
    
    # Compute lost packets based on original sequence number
    actual_loss = count_skips([p[4] for p in packets_out if p[7] and p[4] > -1])

    # Compute lost packets based on rewritten sequence number
    receiver_observed_loss = count_skips([p[5] for p in packets_out if p[7] and p[5] > -1])

    retx_pct = (receiver_observed_loss - actual_loss)*100.0/PACKETS_COUNT
    print(f"Extra retransmissions: {receiver_observed_loss} - {actual_loss} = {receiver_observed_loss - actual_loss} ({round(retx_pct, 4)}%)")
            
    return (receiver_observed_loss - actual_loss)/PACKETS_COUNT

In [ ]:
def report_aggresive_rewrites(packets_out):
    print("Aggresive rewrites:\n")
    count_agg_rewrites = 0
    for gn, fn, ln, sn, rn, srn, mk, pr in packets_out:
        if pr and rn > srn and srn > -1:
            count_agg_rewrites += 1
            print([gn, fn, ln, sn, rn, srn, mk, pr])
    return count_agg_rewrites*100.0/PACKETS_COUNT

In [ ]:
def report_sequence_number_order_violations(packets_out):
    print("Original sequence number order violated:\n")
    packets_ordered_by_rn = sorted([p for p in packets_out if p[5] > -1], key=lambda x: x[4])
    packets_ordered_by_srn = sorted([p for p in packets_out if p[5] > -1], key=lambda x: x[5])
    if packets_ordered_by_rn != packets_ordered_by_srn:
        for p1, p2 in zip(packets_ordered_by_rn, packets_ordered_by_srn):
            if p1 != p2:
                print(f"{p1} compared to {p2}")

In [ ]:
def report_repeats(packets_out):
    print("Repeated rewritten sequence numbers:\n")
    reference_sequence_counts = Counter(p[4] for p in packets_out)
    rewritten_sequence_counts = Counter(p[5] for p in packets_out)
    
    repeated_seqnums_in_reference = [(k, v) for k, v in reference_sequence_counts.items() if k > -1 and v > 1]
    repeated_seqnums_in_rewritten = [(k, v) for k, v in rewritten_sequence_counts.items() if k > -1 and v > 1]
    
    repeats_in_reference = sorted([v for k, v in repeated_seqnums_in_reference])
    repeats_in_rewritten = sorted([v for k, v in repeated_seqnums_in_rewritten])
    
    if repeats_in_reference != repeats_in_rewritten:
        print("Repeats mismatch!")
    
    print("\nRepeats in reference:\n")
    for p in repeated_seqnums_in_reference:
        print(p)
    print("\nRepeats in rewritten:\n")
    for p in repeated_seqnums_in_rewritten:
        print(p)

In [ ]:
def report_mismatches(packets_out):
    print("Mismatches:\n")
    for gn, fn, ln, sn, rn, srn, mk, pr in packets_out:
        if rn != srn and srn > -1:
            print([gn, fn, ln, sn, rn, srn, mk])

In [ ]:
LOSS_PROB = None
REORDER_PROB = 0.0
REORDER_MAX_SKIP = 5
# REORDER_SHUFFLE = False
DROP_WHEN_UNSURE = False

In [ ]:
if LOSS_PROB is None:
    LOSS_PROB_SWEEPSPACE = [i/100 for i in list(range(0, 100, 1))]
    print(f"Loss prob sweep space: {LOSS_PROB_SWEEPSPACE}")
else:
    LOSS_PROB_SWEEPSPACE = []

In [ ]:
SOLUTION_TYPE = 5 # Hardware-amenable (S-LR)

In [ ]:
if LOSS_PROB:
    packets_in, packets_lost, packets_reordered = simulate_loss_and_reordering(LOSS_PROB, REORDER_PROB, REORDER_MAX_SKIP)
    if SOLUTION_TYPE == 4:
        packets_out = solution_type4(packets_in, DROP_WHEN_UNSURE)
    elif SOLUTION_TYPE == 5:
        packets_out = solution_type5(packets_in)
    elif SOLUTION_TYPE == 6:
        packets_out = solution_type6(packets_in, DROP_WHEN_UNSURE)
    elif SOLUTION_TYPE == 2:
        packets_out = solution_type2(packets_in)
    else:
        packets_out = []
    report_sanity_check_results(packets_in, packets_out)
    extra_drop_pct = compute_extra_drops(packets_out)
    extra_retx_pct = compute_extra_retransmissions(packets_out, exit_on_error=False)

In [ ]:
if LOSS_PROB:
    print("\nPackets in:")
    for p in packets_in:
        print(p)

In [ ]:
if LOSS_PROB:
    print("\nPackets out:")
    for p in packets_out:
        print(p)

In [ ]:
if LOSS_PROB:
    report_aggresive_rewrites(packets_out)

In [ ]:
if LOSS_PROB:
    report_sequence_number_order_violations(packets_out)

In [ ]:
if LOSS_PROB:
    report_repeats(packets_out)

In [ ]:
if LOSS_PROB:
    report_mismatches(packets_out)

In [ ]:
extra_drops = []
extra_retxs = []

for loss_prob in LOSS_PROB_SWEEPSPACE:
    packets_in, _, _ = simulate_loss_and_reordering(loss_prob, REORDER_PROB, REORDER_MAX_SKIP)
    
    if SOLUTION_TYPE == 0:
        packets_out = solution_type0(packets_in)
    elif SOLUTION_TYPE == 1:
        packets_out = solution_type1(packets_in)
    elif SOLUTION_TYPE == 2:
        packets_out = solution_type2(packets_in)
    elif SOLUTION_TYPE == 3:
        packets_out = solution_type3(packets_in, DROP_WHEN_UNSURE)
    elif SOLUTION_TYPE == 4:
        packets_out = solution_type4(packets_in, DROP_WHEN_UNSURE)
    elif SOLUTION_TYPE == 5:
        packets_out = solution_type5(packets_in)
    elif SOLUTION_TYPE == 6:
        packets_out = solution_type6(packets_in, DROP_WHEN_UNSURE)

    report_sanity_check_results(packets_in, packets_out)
    extra_drop_pct = compute_extra_drops(packets_out)
    extra_retx_frc = compute_extra_retransmissions(packets_out)
    print()

    extra_drops.append(extra_drop_pct)
    extra_retxs.append(extra_retx_frc)

In [ ]:
print(LOSS_PROB_SWEEPSPACE)

In [ ]:
print(extra_retxs)

In [ ]:
# From some previous reference implementation
reference_loss_prob_sweepspace = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
reference_extra_retx_fractions = [0.0, 0.0002, 0.0022, 0.0079, 0.0128, 0.0185, 0.0274, 0.035, 0.0465, 0.059,
                                  0.0764, 0.0942, 0.1168, 0.1382, 0.1664, 0.2061, 0.2456, 0.3009, 0.3645, 0.4282]

In [ ]:
pu.lineplot(LOSS_PROB_SWEEPSPACE, extra_retxs, {"xlabel": "Loss rate", "ylabel": "Err. retx. rate", "xlim": (-0.05, 1.05)})

In [ ]:
for x, y in zip(LOSS_PROB_SWEEPSPACE, extra_retxs):
    print(f"{x},{y}")

In [ ]:
extra_drops = {"S-LM": [], "S-LR": []}
extra_retxs = {"S-LM": [], "S-LR": []}

for solution_type in ["S-LM", "S-LR"]:
    for loss_prob in LOSS_PROB_SWEEPSPACE:
        packets_in, _, _ = simulate_loss_and_reordering(loss_prob, REORDER_PROB, REORDER_MAX_SKIP)
        
        if solution_type == "S-LM":
            packets_out = scallop_sr_lm(packets_in)
        elif solution_type == "S-LR":
            packets_out = scallop_sr_lr(packets_in)
            
        report_sanity_check_results(packets_in, packets_out)
        extra_drop_pct = compute_extra_drops(packets_out)
        extra_retx_frc = compute_extra_retransmissions(packets_out)
        print()
        
        extra_drops[solution_type].append(extra_drop_pct)
        extra_retxs[solution_type].append(extra_retx_frc)

In [ ]:
pu.lineplots([LOSS_PROB_SWEEPSPACE]*2, [extra_retxs["S-LM"], extra_retxs["S-LR"]],
            {"xlabel": "Loss rate", "ylabel": "Err. retx. rate", "xlim": (-0.05, 1.05),
            "curvelabels": ["S-LM", "S-LR"]})

<b>Notes:</b>
1. <b>Reordering:</b> Trouble when a packet that needs to be dropped intentionally arrives before preceding packets that shouldn't be dropped intentionally.
The drop counter gets incremented too soon and we get the reordered packet's sequence number wrong.
It recovers quickly though, as soon as the packets affected by reordering have passed.